# 03 — Entrenamiento y Evaluación (SUPERVISED y CONTINUAL con EWC/NAIVE)

Este notebook entrena un modelo **SNN** para **regresión del ángulo de dirección (steering)** en dos protocolos:

- **Supervised** sobre `circuito1`.
- **Continual** con dos tareas secuenciales `circuito1 → circuito2` usando:
  - **EWC** (consolidación elástica de pesos), o
  - **NAIVE** (baseline sin penalización; equivalente a λ=0).

> **Requisitos previos**: Ejecuta `01_DATA_QC_PREP.ipynb` para generar `train/val/test.csv` y `tasks.json`.


In [1]:
# =============================================================================
# Imports y setup
# =============================================================================
from pathlib import Path
import sys, json, torch

# Detecta la raíz del repo (si estás dentro de notebooks/, sube un nivel)
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Utilidades y módulos del proyecto
from src.datasets import AugmentConfig
from src.utils import set_seeds, load_preset, make_loaders_from_csvs, ImageTransform
from src.models import SNNVisionRegressor
from src.training import TrainConfig, train_supervised, _permute_if_needed
from src.methods.ewc import EWC, EWCConfig

# Dispositivo (CUDA si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT, device

SEED = 42

# Transformación de imagen
# IMPORTANTE: usa argumentos **posicionales** (w, h, to_gray, crop_top)
# Evita keywords tipo target_w/target_h porque la clase no los define.
tfm = ImageTransform(160, 80, True, None)

def make_snn_model(tfm):
    # C = 1 si gris, 3 si color
    return SNNVisionRegressor(in_channels=(1 if tfm.to_gray else 3), lif_beta=0.95)

torch.set_num_threads(4)               # evita sobre-contención de CPU al decodificar
torch.backends.cudnn.benchmark = True  # selecciona la mejor impl. de convs para tamaño fijo


In [2]:
# --- SAFE MODE: desactiva todo lo pesado para estabilizar ---
SAFE_MODE = False  # o False cuando quieras rendimiento

NUM_WORKERS   = 4
PREFETCH      = 4
PIN_MEMORY    = False
PERSISTENT    = False

# AUG_CFG = None
from src.datasets import AugmentConfig
# --- Augment: perfiles ---
AUG_CFG_LIGHT = AugmentConfig(prob_hflip=0.5, brightness=None, gamma=None, noise_std=0.0)
AUG_CFG_FULL  = AugmentConfig(prob_hflip=0.5, brightness=(0.9, 1.1), gamma=(0.95, 1.05), noise_std=0.005)

AUG_CFG = AUG_CFG_LIGHT   # ← usa LIGHT ahora; luego prueba FULL

USE_OFFLINE_BALANCED  = True
USE_ONLINE_BALANCING  = False

if SAFE_MODE:
    NUM_WORKERS  = 0
    PREFETCH     = None   # ← IMPORTANTÍSIMO con num_workers=0
    PIN_MEMORY   = False
    PERSISTENT   = False  # ← también obligatorio con num_workers=0
    USE_OFFLINE_BALANCED = False
    USE_ONLINE_BALANCING = False
    AUG_CFG = None

print(f"[SAFE_MODE={SAFE_MODE}] workers={NUM_WORKERS} prefetch={PREFETCH} "
      f"pin={PIN_MEMORY} persistent={PERSISTENT}")

RUN_BENCH = False   # pon True para ejecutar

[SAFE_MODE=False] workers=4 prefetch=4 pin=False persistent=False


In [3]:
# =============================================================================
# Verificación de datos (normal y, si existe, balanceado offline)
# =============================================================================
from pathlib import Path

RAW  = ROOT/"data"/"raw"/"udacity"
PROC = ROOT/"data"/"processed"

RUNS = ["circuito1","circuito2"]  # ajusta si hace falta

missing = []
for run in RUNS:
    base = PROC / run

    # Comprobación obligatoria: splits normales
    for part in ["train","val","test"]:
        p = base / f"{part}.csv"
        if not p.exists():
            missing.append(str(p))

    # Comprobación opcional: train_balanced.csv (para modo OFFLINE balanceado)
    p_bal = base / "train_balanced.csv"
    if p_bal.exists():
        print(f"✓ {p_bal} OK")
    else:
        print(f"⚠️  Falta {p_bal}. Si más abajo pones USE_OFFLINE_BALANCED=True, "
              f"ejecuta 01A_PREP_BALANCED.ipynb o el script tools/make_splits_balanced.py")

if missing:
    raise FileNotFoundError(
        "Faltan CSV obligatorios (ejecuta 01A_PREP_BALANCED.ipynb o tu pipeline de prep):\n"
        + "\n".join(" - " + m for m in missing)
    )

print("OK: splits 'train/val/test' encontrados.")


✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito1/train_balanced.csv OK
✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito2/train_balanced.csv OK
OK: splits 'train/val/test' encontrados.


In [4]:
# ===================== Balanceo: helper =====================
print(
    "Modo balanceo:",
    "OFFLINE (tasks_balanced.json)" if USE_OFFLINE_BALANCED else "ORIGINAL (tasks.json)",
    "| Balanceo ONLINE:", USE_ONLINE_BALANCING
)

# Seguridad anti doble balanceo:
if USE_OFFLINE_BALANCED and USE_ONLINE_BALANCING:
    raise RuntimeError("Doble balanceo detectado: OFFLINE y ONLINE a la vez. "
                       "Pon USE_ONLINE_BALANCING=False cuando uses train_balanced.csv.")

from pathlib import Path  # (omite esta línea si ya importaste Path arriba)

def _balance_flag(train_csv_path: str | Path) -> bool:
    """
    Activa balanceo ONLINE solo si:
    - USE_ONLINE_BALANCING == True
    - Y el CSV de train NO es 'train_balanced.csv'
    """
    is_balanced_csv = Path(train_csv_path).name == "train_balanced.csv"
    return bool(USE_ONLINE_BALANCING and not is_balanced_csv)


Modo balanceo: OFFLINE (tasks_balanced.json) | Balanceo ONLINE: False


In [5]:
# =============================================================================
# Función para crear loaders de una tarea dada (respeta cfg del preset)
# =============================================================================
def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed,
                   num_workers=NUM_WORKERS, prefetch_factor=PREFETCH,
                   pin_memory=PIN_MEMORY, persistent_workers=PERSISTENT):
    from pathlib import Path
    name  = task["name"]
    paths = task["paths"]

    pw = persistent_workers and (num_workers > 0)
    pf = prefetch_factor if (num_workers > 0) else None

    return make_loaders_from_csvs(
        base_dir=RAW/name,
        train_csv=Path(paths["train"]),
        val_csv=Path(paths["val"]),
        test_csv=Path(paths["test"]),
        batch_size=batch_size,
        encoder=encoder,
        T=T,
        gain=gain,
        tfm=tfm,
        seed=seed,
        # ---- Parche estabilidad WSL ----
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=pw,
        prefetch_factor=pf,
        aug_train=AUG_CFG,
        balance_train=_balance_flag(paths["train"]),
        balance_bins=21,
    )

print(f"[Loaders] workers={NUM_WORKERS} prefetch={PREFETCH} "
      f"pin={PIN_MEMORY} persistent={PERSISTENT} "
      f"offline_bal={USE_OFFLINE_BALANCED} online_bal={USE_ONLINE_BALANCING}")


[Loaders] workers=4 prefetch=4 pin=False persistent=False offline_bal=True online_bal=False


In [6]:
from src.training import _permute_if_needed  # importa el helper del modelo

# =============================================================================
# Helper de evaluación (permuta a (T,B,C,H,W) y usa copias no bloqueantes)
# =============================================================================
def eval_loader(loader, model, device):
    """Calcula MAE/MSE promediados sobre todo el loader.

    - El DataLoader produce (B, T, C, H, W)
    - El modelo espera      (T, B, C, H, W)
    """
    model.eval()  # modo evaluación: desactiva dropout/batchnorm, etc.
    mae_sum = 0.0
    mse_sum = 0.0
    n = 0

    # Un solo no_grad() fuera del bucle para minimizar overhead
    with torch.no_grad():
        for x, y in loader:
            # (B,T,C,H,W) -> (T,B,C,H,W), y luego a GPU con non_blocking=True
            x = _permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            y_hat = model(x)

            # Acumula MAE/MSE ponderados por el tamaño real del batch
            mae_sum += torch.mean(torch.abs(y_hat - y)).item() * len(y)
            mse_sum += torch.mean((y_hat - y) ** 2).item() * len(y)
            n += len(y)

    return (mae_sum / max(n, 1)), (mse_sum / max(n, 1))


In [7]:
# =============================================================================
# Elegir split: normal (tasks.json) o balanceado offline (tasks_balanced.json)
# =============================================================================
with open(PROC / ("tasks_balanced.json" if USE_OFFLINE_BALANCED else "tasks.json"), "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

# Vista rápida: muestra el CSV de train que se usará por cada tarea
print("Tareas y su TRAIN CSV:")
for t in task_list:
    print(f" - {t['name']}: {Path(t['paths']['train']).name}")

task_list[:2]  # vista rápida

# Guardarraíl extra: si has activado el OFFLINE balanceado,
# exige que el 'train' sea train_balanced.csv y que exista.
if USE_OFFLINE_BALANCED:
    for t in task_list:
        train_path = Path(t["paths"]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' pero encontré '{train_path.name}'. "
                "Repite 01A_PREP_BALANCED.ipynb o ajusta USE_OFFLINE_BALANCED=False."
            )
        if not train_path.exists():
            raise FileNotFoundError(
                f"[{t['name']}] No existe {train_path}. Genera los balanceados con 01A_PREP_BALANCED.ipynb."
            )
    print("✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).")


Tareas y su TRAIN CSV:
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).


In [8]:
# ===================== BENCH: toggle y eco de configuración =====================
# Usa el RUN_BENCH que ya defines en la Celda 2
print(
    f"[Bench RUN_BENCH={RUN_BENCH}] workers={NUM_WORKERS} prefetch={PREFETCH} "
    f"pin={PIN_MEMORY} persistent={PERSISTENT} "
    f"| offline_bal={USE_OFFLINE_BALANCED} online_bal={USE_ONLINE_BALANCING}"
)


[Bench RUN_BENCH=False] workers=4 prefetch=4 pin=False persistent=False | offline_bal=True online_bal=False


In [9]:
# ===================== BENCH: utilidades (no pisa tus helpers) =====================
import time, torch
from time import perf_counter
from contextlib import nullcontext
from pathlib import Path

def _bench_smoke_loader(task, bs, encoder, T, gain, tfm, seed):
    """Crea loader pequeño y mide el primer batch."""
    tr, va, te = make_loader_fn(
        task=task, batch_size=bs, encoder=encoder, T=T, gain=gain, tfm=tfm, seed=seed,
        num_workers=NUM_WORKERS, prefetch_factor=PREFETCH,
        pin_memory=PIN_MEMORY, persistent_workers=PERSISTENT
    )
    t0 = perf_counter()
    _ = next(iter(tr))
    dt = perf_counter() - t0
    return tr, va, te, dt

def _bench_forward_once(model, xb, use_amp=True):
    """Mide un forward único (ms). xb: (T,B,C,H,W) ya en device."""
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    ctx = torch.amp.autocast('cuda', enabled=use_amp) if torch.cuda.is_available() else nullcontext()
    with torch.inference_mode(), ctx:
        _ = model(xb)
    torch.cuda.synchronize()
    return (time.perf_counter() - t0) * 1000.0

def _bench_loop_gpu_only(model, xb, iters=100, use_amp=True):
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    ctx = torch.amp.autocast('cuda', enabled=use_amp) if torch.cuda.is_available() else nullcontext()
    with torch.inference_mode(), ctx:
        for _ in range(iters):
            _ = model(xb)
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    return iters / (t1 - t0)

def _bench_pipeline_loader_model(model, tr, iters=100, use_amp=True, device=None):
    it_tr = iter(tr)
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    ctx = torch.amp.autocast('cuda', enabled=use_amp) if torch.cuda.is_available() else nullcontext()
    with torch.inference_mode(), ctx:
        for _ in range(iters):
            try:
                xb, yb = next(it_tr)
            except StopIteration:
                it_tr = iter(tr)
                xb, yb = next(it_tr)
            xb = xb.permute(1,0,2,3,4).contiguous().to(device, non_blocking=True)
            _ = model(xb)
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    return iters / (t1 - t0)


In [10]:
# ===================== BENCH: ejecución unificada =====================
if not RUN_BENCH:
    print("[Bench desactivado]")
else:
    from src.utils import load_preset

    # --- preset y tarea de referencia (como en tu entrenamiento) ---
    task0 = task_list[0]
    preset_name = "std"  # cambia a "fast" si quieres medir ese preset
    cfgp = load_preset(ROOT/"configs"/"presets.yaml", preset_name)

    B = int(cfgp["batch_size"])
    T = int(cfgp["T"])
    GAIN = float(cfgp["gain"])
    ENCODER = cfgp["encoder"]
    USE_AMP = bool(cfgp["amp"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- 1) Smoke test del loader (batch mini) ---
    tr_mini, _, _, t_first = _bench_smoke_loader(task0, bs=4, encoder=ENCODER, T=5, gain=0.5, tfm=tfm, seed=SEED)
    print(f"Primer batch pequeño: {t_first:.3f}s")

    # --- 2) Loader real de preset + modelo (usa tu helper make_snn_model) ---
    tr_big, _, _ = make_loader_fn(
        task=task0, batch_size=B, encoder=ENCODER, T=T, gain=GAIN, tfm=tfm, seed=SEED,
        num_workers=NUM_WORKERS, prefetch_factor=PREFETCH,
        pin_memory=PIN_MEMORY, persistent_workers=PERSISTENT
    )
    xb, yb = next(iter(tr_big))                         # (B,T,C,H,W)
    xb = xb.permute(1,0,2,3,4).contiguous().to(device, non_blocking=True)

    model = make_snn_model(tfm).to(device).eval()

    # --- 3) Forward único (ms) ---
    ms = _bench_forward_once(model, xb, use_amp=USE_AMP)
    print(f"Preset {preset_name} | B={B}, T={T}, AMP={USE_AMP} | Forward 1x: {ms:.1f} ms")

    # --- 4) GPU-only it/s (techo computacional) ---
    its_gpu = _bench_loop_gpu_only(model, xb, iters=100, use_amp=USE_AMP)
    print(f"[GPU-only] 100 iters -> {its_gpu:.1f} it/s")

    # --- 5) Loader+modelo it/s (pipeline real) ---
    its_pipe = _bench_pipeline_loader_model(model, tr_big, iters=100, use_amp=USE_AMP, device=device)
    print(f"[Loader+modelo] 100 iters -> {its_pipe:.1f} it/s")

    # --- 6) Grid corto de (workers, prefetch) para tu preset real ---
    combos = [(4,2), (4,4), (6,2), (6,4), (8,2), (8,4), (12,2)]
    print("\nGrid (workers, prefetch):")
    best = None
    for nw, pf in combos:
        tr_try, _, _ = make_loader_fn(
            task=task0, batch_size=B, encoder=ENCODER, T=T, gain=GAIN, tfm=tfm, seed=SEED,
            num_workers=nw, prefetch_factor=pf, pin_memory=PIN_MEMORY, persistent_workers=PERSISTENT
        )
        ips = _bench_pipeline_loader_model(model, tr_try, iters=80, use_amp=USE_AMP, device=device)
        print(f"  nw={nw:>2} pf={pf:>2} -> {ips:5.1f} it/s")
        if best is None or ips > best[2]:
            best = (nw, pf, ips)

    if best:
        print(f"\nMejor combinación observada: workers={best[0]} prefetch={best[1]} -> {best[2]:.2f} it/s")


[Bench desactivado]


In [11]:
# ====================== run_continual que carga el preset internamente ======================
from pathlib import Path
import json, torch
from src.utils import load_preset, set_seeds
from src.training import TrainConfig, train_supervised
from src.models import SNNVisionRegressor
from src.methods.ewc import EWC, EWCConfig

def run_continual(
    preset: str,                  # "fast" | "std" | "accurate"
    method: str,                  # "ewc" | "naive"
    lam: float | None,            # λ si EWC; None si naive
    seed: int,
    encoder: str,                 # "rate" | "latency"
    tfm,                          # ImageTransform a usar
    fisher_batches_by_preset: dict[str,int] | None = None,
):
    # Carga del preset específico para ESTE run
    cfg = load_preset(ROOT/"configs"/"presets.yaml", preset)
    T      = int(cfg["T"])
    gain   = float(cfg["gain"])
    lr     = float(cfg["lr"])
    epochs = int(cfg["epochs"])
    bs     = int(cfg["batch_size"])
    use_amp= bool(cfg["amp"])

    # Fisher batches según preset (si no se da, usa 100 por defecto)
    fb = 100
    if fisher_batches_by_preset and preset in fisher_batches_by_preset:
        fb = int(fisher_batches_by_preset[preset])

    set_seeds(seed)

    # Modelo y método
    model = make_snn_model(tfm)
    ewc = None
    if method == "ewc":
        assert lam is not None, "Para EWC debes pasar λ (lam)"
        ewc = EWC(model, EWCConfig(lambd=float(lam), fisher_batches=fb))

    # Tareas (asumimos task_list ya existe en el notebook)
    out_tag = f"continual_{preset}_{method}" + (f"_lam_{lam:.0e}" if method=="ewc" else "") + f"_{encoder}_seed_{seed}"
    out_dir = ROOT/"outputs"/out_tag
    out_dir.mkdir(parents=True, exist_ok=True)

    # entrenamiento secuencial
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_fn = torch.nn.MSELoss()
    tcfg = TrainConfig(epochs=epochs, batch_size=bs, lr=lr, amp=use_amp, seed=seed)

    results = {}
    seen = []

    for i, t in enumerate(task_list):
        name = t["name"]

        # ⬇️ AHORA usamos SIEMPRE tu helper de la celda 4
        tr, va, te = make_loader_fn(
            task=t,
            batch_size=bs,
            encoder=encoder,
            T=T,
            gain=gain,
            tfm=tfm,
            seed=seed,
        )

        _ = train_supervised(
            model, tr, va, loss_fn, tcfg,
            out_dir/f"task_{i+1}_{name}",
            method=ewc if method=="ewc" else None
        )

        if method=="ewc":
            print("Estimando Fisher…")
            ewc.estimate_fisher(va, loss_fn, device=device)

        # evaluación tarea actual
        te_mae, te_mse = eval_loader(te, model, device)
        results[name] = {"test_mae": te_mae, "test_mse": te_mse}
        seen.append((name, te))

        # reevaluación tareas previas (olvido)
        for pname, p_loader in seen[:-1]:
            p_mae, p_mse = eval_loader(p_loader, model, device)
            results[pname][f"after_{name}_mae"] = p_mae
            results[pname][f"after_{name}_mse"] = p_mse

    (out_dir/"continual_results.json").write_text(json.dumps(results, indent=2), encoding="utf-8")
    return out_dir, results


In [12]:
# ===================== Demo opcional (1 corrida) =====================
RUN_DEMO = False  # pon True para probar 1 run rápido

if RUN_DEMO:
    preset_demo  = "fast"
    method_demo  = "ewc"     # "ewc" | "naive"
    lam_demo     = 1e9       # si method_demo=="ewc"
    seed_demo    = 42

    demo_cfg = load_preset(ROOT / "configs" / "presets.yaml", preset_demo)
    encoder_demo = demo_cfg["encoder"]   # "rate" o "latency"

    out_path, res = run_continual(
        preset=preset_demo,
        method=method_demo,
        lam=(lam_demo if method_demo == "ewc" else None),
        seed=seed_demo,
        encoder=encoder_demo,
        tfm=tfm,
        fisher_batches_by_preset={"fast": 200, "std": 200},
    )
    print("OK:", out_path)
    res


In [13]:
# ====================== CONFIG POR DEFECTO PARA LAS COMPARATIVAS ======================
# Recomendaciones acordadas:
# - fast: EWC λ=1e9 (estable). Extra: λ=1e8 (mejor T2, algo más de olvido)
# - std : EWC λ=1e7 (baseline estable). Extra: λ=3e7 (mejor T2, más olvido)

EWC_DEFAULTS = {
    "fast":     {"primary": [1e9, 3e8],      "extra": [3e9]},
    "std":      {"primary": [3e8, 1e8],      "extra": [1e9]},
    "accurate": {"primary": [3e7, 1e8],      "extra": [3e8]},  # ← mucho más bajo que 1e9
}

INCLUDE_NAIVE    = True          # añade baseline sin EWC
INCLUDE_EXTRAS   = True          # activa los λ "extra" por preset
SEEDS            = [42, 43, 44]  # multisemillas para medias/σ
ENCODERS         = ["rate"]      # luego podrás añadir "latency"
# FISHER_BY_PRESET = {"fast": 200, "std": 600, "accurate": 600}  # estabiliza el cálculo de Fisher
# FISHER_BY_PRESET = {"fast": 800, "std": 1000}  # estabiliza el cálculo de Fisher
# Endurecemos Fisher para estabilidad; accurate necesita más por ser más largo
FISHER_BY_PRESET = {"fast": 800, "std": 1200, "accurate": 1500}

# Elige qué presets lanzar
PRESETS_TO_RUN = ["fast", "std", "accurate"]  # añade "accurate" si lo necesitas más adelante
# PRESETS_TO_RUN = ["accurate"]  # añade "accurate" si lo necesitas más adelante

# ---- Construcción del plan de ejecuciones ----
runs_plan = []
for preset_i in PRESETS_TO_RUN:
    # EWC primary
    for lam in EWC_DEFAULTS[preset_i]["primary"]:
        runs_plan.append((preset_i, "ewc", lam))
    # EWC extras (opcionales)
    if INCLUDE_EXTRAS:
        for lam in EWC_DEFAULTS[preset_i]["extra"]:
            runs_plan.append((preset_i, "ewc", lam))
    # Baseline sin EWC
    if INCLUDE_NAIVE:
        runs_plan.append((preset_i, "naive", None))

print("Plan de runs (preset, método, λ):")
for preset_i, method_i, lam_i in runs_plan:
    print(f"  {preset_i:>7}  {method_i:>5}  λ={lam_i}")
print("Semillas:", SEEDS, " | Encoders:", ENCODERS)
print("Fisher batches por preset:", FISHER_BY_PRESET)


Plan de runs (preset, método, λ):
     fast    ewc  λ=1000000000.0
     fast    ewc  λ=300000000.0
     fast    ewc  λ=3000000000.0
     fast  naive  λ=None
      std    ewc  λ=300000000.0
      std    ewc  λ=100000000.0
      std    ewc  λ=1000000000.0
      std  naive  λ=None
  accurate    ewc  λ=30000000.0
  accurate    ewc  λ=100000000.0
  accurate    ewc  λ=300000000.0
  accurate  naive  λ=None
Semillas: [42, 43, 44]  | Encoders: ['rate']
Fisher batches por preset: {'fast': 800, 'std': 1200, 'accurate': 1500}


In [ ]:
# ====================== DRIVER MULTISEMILLAS (usa la CONFIG de arriba) ======================
for enc in ENCODERS:
    for seed in SEEDS:
        for preset_i, method_i, lam_i in runs_plan:
            print(f"\n=== RUN: preset={preset_i} | method={method_i} | λ={lam_i} | seed={seed} | encoder={enc} ===")
            out_path, _ = run_continual(
                preset=preset_i,
                method=method_i,
                lam=(lam_i if method_i == "ewc" else None),
                seed=seed,
                encoder=enc,
                tfm=tfm,  # definido en tu celda de setup
                fisher_batches_by_preset=FISHER_BY_PRESET,
            )
            print("OK:", out_path)
print("\n✅ Listo. Ejecuta las celdas de resumen.")



=== RUN: preset=fast | method=ewc | λ=1000000000.0 | seed=42 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_1e+09_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=300000000.0 | seed=42 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_3e+08_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=3000000000.0 | seed=42 | encoder=rate ===


...

Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_ewc_lam_1e+08_rate_seed_42

=== RUN: preset=std | method=ewc | λ=1000000000.0 | seed=42 | encoder=rate ===


Estimando Fisher…


Estimando Fisher…
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_ewc_lam_1e+09_rate_seed_42

=== RUN: preset=std | method=naive | λ=None | seed=42 | encoder=rate ===


OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_std_naive_rate_seed_42

=== RUN: preset=accurate | method=ewc | λ=30000000.0 | seed=42 | encoder=rate ===


Epoch 12/20:  96%|█████████▌| 873/909 [00:58<00:02, 15.63it/s]

In [ ]:
# =============================================================================
# Resumen comparativo de todos los continual_* en outputs/
# =============================================================================
import re, json
from pathlib import Path
import pandas as pd

def parse_exp_name(name: str):
    """
    Extrae preset, método, lambda, encoder y seed del nombre de carpeta:

      continual_<preset>_<method>[_lam_<lambda>]_<_encoder>[_seed_<seed>]

    Ejemplos:
      continual_fast_naive_rate_seed_42
      continual_fast_ewc_lam_1e+08_rate_seed_42
      continual_std_ewc_lam_3e+07_latency_seed_43
    """
    m = re.match(
        r"continual_(?P<preset>\w+)_(?P<method>ewc|naive)"
        r"(?:_lam_(?P<lambda>[^_]+))?_(?P<enc>[^_]+)"
        r"(?:_seed_(?P<seed>\d+))?$",
        name
    )
    meta = {"preset": None, "method": None, "lambda": None, "encoder": None, "seed": None}
    if m:
        d = m.groupdict()
        meta.update({
            "preset": d["preset"],
            "method": d["method"],
            "lambda": d.get("lambda"),
            "encoder": d.get("enc"),
            "seed": d.get("seed"),
        })
    return meta

rows = []
root_out = ROOT / "outputs"

for exp_dir in sorted(root_out.glob("continual_*")):
    name = exp_dir.name
    meta = parse_exp_name(name)

    # Saltar nombres no reconocidos (runs muy antiguos)
    if meta["preset"] is None:
        continue

    results_path = exp_dir / "continual_results.json"
    if not results_path.exists():
        continue

    with open(results_path, "r", encoding="utf-8") as f:
        res = json.load(f)

    # Detectar tareas: la "última" es la que NO tiene claves 'after_*'
    task_names = list(res.keys())
    if len(task_names) < 2:
        continue

    def is_last(d):  # no tiene after_*
        return not any(k.startswith("after_") for k in d.keys())

    last_task = None
    first_task = None
    for tn in task_names:
        if is_last(res[tn]):
            last_task = tn
        else:
            first_task = tn

    # Fallback por si no se identifica bien
    if first_task is None or last_task is None:
        task_names_sorted = sorted(task_names)
        first_task = task_names_sorted[0]
        last_task  = task_names_sorted[-1]

    c1, c2 = first_task, last_task

    c1_test_mae = float(res[c1].get("test_mae", float("nan")))
    c2_test_mae = float(res[c2].get("test_mae", float("nan")))
    after_key_mae = f"after_{c2}_mae"
    c1_after_c2_mae = float(res[c1].get(after_key_mae, float("nan")))

    forgetting_abs = c1_after_c2_mae - c1_test_mae
    forgetting_rel = (forgetting_abs / c1_test_mae * 100.0) if c1_test_mae == c1_test_mae else float("nan")

    rows.append({
        "exp": name,
        "preset": meta["preset"],
        "method": meta["method"],
        "lambda": meta["lambda"] if meta["method"] == "ewc" else None,
        "encoder": meta["encoder"],
        "seed": int(meta["seed"]) if meta["seed"] is not None else None,
        "c1_name": c1,
        "c2_name": c2,
        "c1_mae": c1_test_mae,
        "c1_after_c2_mae": c1_after_c2_mae,
        "c1_forgetting_mae_abs": forgetting_abs,
        "c1_forgetting_mae_rel_%": forgetting_rel,
        "c2_mae": c2_test_mae,
    })

df = pd.DataFrame(rows)

# Asegura columnas numéricas auxiliares
if "lambda_num" not in df.columns:
    df["lambda_num"] = pd.to_numeric(df["lambda"], errors="coerce")  # '1e+08' -> 1e+08 ; NAIVE -> NaN

# Deja 'seed' como entero y elimina 'seed_num' si existe
df["seed"] = pd.to_numeric(df["seed"], errors="coerce").astype("Int64")
if "seed_num" in df.columns:
    df = df.drop(columns=["seed_num"])

# Ordenar: preset, method, encoder, lambda_num (NaN al final), seed
df = df.sort_values(
    by=["preset", "method", "encoder", "lambda_num", "seed"],
    na_position="last",
    ignore_index=True,
)

df


In [ ]:
# ====================== Vista agregada (media±std por preset/method/λ/encoder) ======================
import pandas as pd

# Métricas a agregar
cols_metrics = ["c1_mae", "c1_after_c2_mae", "c1_forgetting_mae_abs", "c1_forgetting_mae_rel_%", "c2_mae"]

# Copia y asegura columna numérica auxiliar para ordenar por λ
gdf = df.copy()
if "lambda_num" not in gdf.columns:
    gdf["lambda_num"] = pd.to_numeric(gdf["lambda"], errors="coerce")  # NA para NAIVE

# Agregación: media, std y número de corridas (semillas) por combinación
agg = (
    gdf
    .groupby(["preset", "method", "encoder", "lambda", "lambda_num"], dropna=False)[cols_metrics]
    .agg(["mean", "std", "count"])
    .reset_index()
)

# Aplanar nombres de columnas (de MultiIndex a una sola capa)
agg.columns = [
    "_".join(filter(None, map(str, col))).rstrip("_")
    for col in agg.columns.to_flat_index()
]

# Ordena por preset/method/encoder/λ_num (NaN al final ⇒ NAIVE al final de su grupo)
agg = agg.sort_values(
    by=["preset", "method", "encoder", "lambda_num"],
    na_position="last",
    ignore_index=True,
)

# (Opcional) guardar a CSV
summary_dir = ROOT / "outputs" / "summary"
summary_dir.mkdir(parents=True, exist_ok=True)
agg.to_csv(summary_dir / "continual_summary_agg.csv", index=False)
print("Guardado:", summary_dir / "continual_summary_agg.csv")

agg


In [ ]:
# ====================== Formateo para la memoria (tabla compacta) ======================

def fmt(x, prec=4):
    # Redondea y gestiona NaN de forma amigable
    import pandas as pd
    return "" if pd.isna(x) else f"{x:.{prec}f}"

show = agg.copy()

# 1) Crea 'count' a partir de cualquiera de las columnas *_count
count_cols = [c for c in show.columns if c.endswith("_count")]
if count_cols:
    show["count"] = show[count_cols[0]].astype("Int64")  # todas deberían coincidir
    # (opcional) elimina las columnas *_count individuales
    show = show.drop(columns=count_cols)

# 2) Redondea columnas de medias/desviaciones
for c in [c for c in show.columns if c.endswith("_mean") or c.endswith("_std")]:
    show[c] = show[c].map(lambda v: fmt(v, 4))

# 3) Selección de columnas clave (ajusta el orden a tu gusto)
cols = [
    "preset", "method", "encoder", "lambda",
    "c1_mae_mean", "c1_forgetting_mae_rel_%_mean", "c2_mae_mean",
    "c1_mae_std",  "c1_forgetting_mae_rel_%_std",  "c2_mae_std",
    "count"
]

# Si alguna columna no existiera (según tus métricas), la ignoramos con aviso
missing = [c for c in cols if c not in show.columns]
if missing:
    print("Aviso: faltan columnas en 'show':", missing)
    cols = [c for c in cols if c in show.columns]

show = show[cols].rename(columns={
    "preset": "preset",
    "method": "método",
    "encoder": "codificador",
    "lambda": "λ",
    "c1_mae_mean": "MAE Tarea1 (media)",
    "c1_forgetting_mae_rel_%_mean": "Olvido T1 (%) (media)",
    "c2_mae_mean": "MAE Tarea2 (media)",
    "c1_mae_std": "MAE Tarea1 (σ)",
    "c1_forgetting_mae_rel_%_std": "Olvido T1 (%) (σ)",
    "c2_mae_std": "MAE Tarea2 (σ)",
    "count": "n (semillas)"
})

show
